In [1]:
from huggingface_hub import login
login('')

/home/jovyan/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.data_utils import SIFT50MDataset
import os
from datasets import load_dataset, Dataset

sift_dataset = load_dataset(
    'amazon-agi/SIFT-50M',
    name='closed_ended_comparison',
    split='train',
    trust_remote_code=True
)
sift_dataset = sift_dataset.shuffle(seed=50)
df = sift_dataset.select(range(100)).to_pandas()
print(len(sift_dataset))
allowed_values = ["common_voice_de", "multilingual_librispeech_de", "common_voice_en",'vctk_en'] # The datasets were taking too long to get donwloaded so I restricted it to certain subsets

filtered_df = df[df["data_source"].isin(allowed_values)]

# Count number of entries per value
counts = filtered_df["data_source"].value_counts()
print(counts)
sift_data = Dataset.from_pandas(filtered_df)
# Define the base datasets paths (replace with your actual paths)


3613791
data_source
common_voice_en                11
multilingual_librispeech_de     6
common_voice_de                 1
Name: count, dtype: int64


In [53]:
filtered_df.iloc[100].messages

array([{'role': 'user', 'content': array([{'text': 'Which audio clip mentions a character from a series?', 'audio_path': None},
              {'text': None, 'audio_path': '/path/to/common_voice_en_22177961.wav'},
              {'text': None, 'audio_path': '/path/to/common_voice_en_22462206.wav'}],
             dtype=object)}                                                                                                    ,
       {'role': 'assistant', 'content': array([{'text': 'Audio clip 2 mentions a character from a series.', 'audio_path': None}],
             dtype=object)}                                                                                                      ],
      dtype=object)

In [3]:
filtered_df_1 = filtered_df[filtered_df["data_source"].isin(['multilingual_librispeech_de'])]
print(filtered_df_1.iloc[1])

id                 comparison_2422_2340_009836__2422_3333_007576
messages       [{'role': 'user', 'content': [{'text': 'Wie is...
task                                     closed_ended_comparison
data_source                          multilingual_librispeech_de
Name: 7, dtype: object


In [4]:
sift_data = Dataset.from_pandas(filtered_df_1)


In [ ]:
base_datasets_root = "/home/jovyan/.cache/huggingface/datasets"
base_datasets_paths = {
   # "common_voice_de": None, # No longer needs a path, handled by load_dataset
    "multilingual_librispeech_de": None, # No longer needs a path, handled by load_dataset
   # "common_voice_en": None, # No longer needs a path, handled by load_dataset
   # "vctk_en": "./vctk_corpus" # VCTK still needs a root path for torchaudio
}

# Create an instance of the SIFT50MDataset
sift_iterable_dataset = SIFT50MDataset(sift_dataset=sift_data.select(range(1)), base_datasets_paths=base_datasets_paths)

# Iterate through the dataset and print the audio path and some metadata
print("Iterating through SIFT-50M dataset:")
for i, data in enumerate(sift_iterable_dataset):
    if i >= 5:  # Print first 5 examples for demonstration
        break
    print(f"Example {i+1}:")
    print(f"  Audio Path: {data['audio_path']}")
    print(f"  Data Source: {data['metadata']['data_source']}")
    print(f"  SIFT ID: {data['metadata']['id']}")
    print(f"  full message: {data}")
    print("\n")

print("Demonstration complete.")

multilingual_librispeech_de None
Iterating through SIFT-50M dataset:
multilingual_librispeech_de {'id': 'comparison_2422_2200_000717__2422_3059_000247', 'messages': [{'content': [{'audio_path': None, 'text': 'Wie ist die Sprachqualität in den beiden Aufnahmen?'}, {'audio_path': '/path/to/2422_2200_000717.wav', 'text': None}, {'audio_path': '/path/to/2422_3059_000247.wav', 'text': None}], 'role': 'user'}, {'content': [{'audio_path': None, 'text': 'Die erste Aufnahme hat eine großartige Sprachqualität, während die zweite eine mäßige Sprachqualität aufweist.'}], 'role': 'assistant'}], 'task': 'closed_ended_comparison', 'data_source': 'multilingual_librispeech_de', '__index_level_0__': 5}
filtering for id


Filter:   1%|▏         | 7000/469942 [04:03<4:14:45, 30.29 examples/s]Exception ignored from cffi callback <function SoundFile._init_virtual_io.<locals>.vio_read at 0x7f89455eb100>:
Traceback (most recent call last):
  File "/home/jovyan/.local/opt/conda/envs/ainmc/lib/python3.11/site-packages/soundfile.py", line 1290, in vio_read
    @_ffi.callback("sf_vio_read")

KeyboardInterrupt: 
Filter:  11%|█         | 50000/469942 [29:04<4:02:39, 28.84 examples/s]

In [ ]:
?os.path.splitext